In [1]:
import os

import torch
from datasets import load_dataset
from transformers import TrainingArguments

while "notebooks" in os.getcwd():
    os.chdir("..")

from mim_nlp.seq2seq import AutoSummarizer

/home/jupyter-mbrzozowski/.conda/envs/mim_nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Choose GPU

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
torch.cuda.is_available()

True

# Loading an open-source summary dataset

In [3]:
dataset = load_dataset("allegro/summarization-polish-summaries-corpus")

Found cached dataset csv (/home/jupyter-mbrzozowski/.cache/huggingface/datasets/allegro___csv/allegro--summarization-polish-summaries-corpus-85a9d1415b78fdd5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 574.35it/s]


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 15598
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 1734
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 4334
    })
})

In [5]:
dataset["train"]["target"][0]

'We wrześniu w życie wejdzie reforma oświaty, więc niezbędna będzie wymiana wielu podręczników. Wykaz książek dopuszczonych przez MEN do nauki w szkołach ukaże się dopiero pod koniec sierpnia. Wydawnictwa szukają sposobów, by zachęcić nauczycieli do wybrania ich podręczników. Do tej pory nauczyciele niechętnie zmieniali podręczniki, z których uczyli, a teraz będą do tego zmuszeni.'

In [6]:
len(dataset["train"]["source"])

15598

# Training a model on the dataset sample

In [7]:
training_arguments = TrainingArguments(
    output_dir="tmp", num_train_epochs=1, logging_steps=100, per_device_train_batch_size=2, optim="adamw_torch"
)

In [8]:
# to use this model, install protobuf==3.20 and sentencepiece
model = AutoSummarizer(
    pretrained_model_name_or_path="allegro/plt5-base", max_length=1024, training_args=training_arguments
)

In [9]:
N_SAMPLES = 1000
model.fit(dataset["train"]["source"][:N_SAMPLES], dataset["train"]["target"][:N_SAMPLES])

Step,Training Loss
100,7.082500
200,4.031500
300,3.703300
400,3.635500
500,3.635000


# Predicting

In [10]:
source = dataset["validation"]["source"][0]
target = dataset["validation"]["target"][0]

In [11]:
print(target)

Andrzej Gołota przegrał z Michaelem Grantem w 10. rundzie. 27-letni Michael Grant był faworytem w pojedynku z Gołotą. Olbrzym z Norristown (202 cm, 114 kg) nie przegrał żadnej z 30 zawodowych walk. Kilku znanych mistrzów boksu twierdziło jednak publicznie, że Polak ma duże szanse. 
Pierwsze rundy walki w Atlantic City potwierdziły te opinie. Przed dziesiątym starciem Gołota prowadził na punkty u wszystkich sędziów. W dziesiątej rundzie po lewym sierpowym i prawym podbródkowym Granta padł na deski.  Wstał wprawdzie dość szybko, ale Randy Naumann przerwał pojedynek.
Gołota przegrał i zarobił kolejny milion dolarów. Raz jeszcze pokazał, że jest bokserem wyśmienitym, ale charakteru do wygrywania mu brak.


In [12]:
model.predict([source], max_new_tokens=20)

predicting in batches of size 1: 100%|██████████| 1/1 [00:00<00:00,  3.16it/s]


array(['en. W pierwszej rundzie walki z Gołota i Granta były pięściarze, które przegrało w'],
      dtype='<U82')

# Saving the model

In [13]:
model.save("models/my_plt5_model")

# Loading the model

In [14]:
model = AutoSummarizer.load("models/my_plt5_model/")

In [15]:
model

In [16]:
model.predict([source], max_new_tokens=20)

predicting in batches of size 1: 100%|██████████| 1/1 [00:00<00:00,  3.28it/s]


array(['Anity nie było to już tak zdziwieniam. Granie z Gołotą był wy'],
      dtype='<U61')